## Latent regression

In [38]:
import json
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Load the JSON data
with open('results/latent_regression_results.json', 'r') as f:
    data = json.load(f)

# Create subplots
fig = make_subplots(rows=1, cols=1, shared_xaxes=True)

# Colors for each model
colors = {'MLP': 'rgb(31, 119, 180)', 'SparseAutoEncoder': 'rgb(255, 127, 14)'}

max_flops_per_model = []

for model_results in data['results']:
    model_name = model_results['model']
    
    # Collect data from all runs
    all_flops = []
    all_mccs = []
    
    for run in model_results['runs']:
        total_flops = [log['total_flops'] for log in run['performance_log']]
        test_mccs = [log['test_mcc'] for log in run['performance_log']]
        
        # Add 0 FLOP-MCC point
        total_flops.insert(0, 0)
        test_mccs.insert(0, 0)  # Assuming MCC is 0 at the start
        
        all_flops.append(total_flops)
        all_mccs.append(test_mccs)
    
    # Calculate mean and standard deviation
    mean_flops = np.mean(all_flops, axis=0)
    mean_mccs = np.mean(all_mccs, axis=0)
    std_mccs = np.std(all_mccs, axis=0)
    
    # Store the maximum FLOP value for this model
    max_flops_per_model.append(np.max(mean_flops))
    
    # Add trace for each model
    fig.add_trace(
        go.Scatter(
            x=mean_flops,
            y=mean_mccs,
            mode='lines',
            name=f"{model_name}",
            line=dict(color=colors[model_name], width=3),
        )
    )
    
    # Add error bands
    fig.add_trace(
        go.Scatter(
            x=np.concatenate([mean_flops, mean_flops[::-1]]),
            y=np.concatenate([mean_mccs + std_mccs, (mean_mccs - std_mccs)[::-1]]),
            fill='toself',
            fillcolor=colors[model_name].replace('rgb', 'rgba').replace(')', ',0.2)'),
            line=dict(color='rgba(255,255,255,0)'),
            hoverinfo="skip",
            showlegend=False
        )
    )

# Calculate the minimum of the maximum FLOP values
x_max = min(max_flops_per_model)

# Update layout
fig.update_layout(
    xaxis_title='Total FLOPs',
    yaxis_title='Test MCC',
    font=dict(family="Palatino", size=18),
    legend=dict(
        x=0.01,
        y=0.99,
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='rgba(0, 0, 0, 0.1)',
        borderwidth=1,
        font=dict(size=16)
    ),
    plot_bgcolor='white',
    hovermode='x unified',
    width=800,
    height=500,
    margin=dict(l=80, r=50, t=100, b=80)
)

# Update axes
fig.update_xaxes(
    showgrid=True,
    gridwidth=1,
    gridcolor='rgba(0, 0, 0, 0.1)',
    zeroline=False,
    showline=True,
    linewidth=2,
    linecolor='black',
    type='log',
    title_standoff=20,
    #range=[0, np.log10(x_max)]  # Set x-axis range
)

fig.update_yaxes(
    showgrid=True,
    gridwidth=1,
    gridcolor='rgba(0, 0, 0, 0.1)',
    zeroline=False,
    showline=True,
    linewidth=2,
    linecolor='black',
    range=[0, 1],
    title_standoff=20
)

# Save the plot as a PDF
fig.write_image("results/figures/latent_regression_results.pdf")

# Display the plot
fig.show()

print("Plot saved as results/figures/latent_regression_results.pdf")

Plot saved as results/figures/latent_regression_results.pdf


## Fixed dictionary vectors

In [43]:
import json
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

FIXED_D = True  # Set this to True or False based on your experiment
filename = 'fixed_D' if FIXED_D else 'learned_D'

# Load the JSON data
with open(f'results/{filename}_results.json', 'r') as f:
    data = json.load(f)

# Create subplots
fig = make_subplots(rows=1, cols=1, shared_xaxes=True)

# Colors for each model
colors = {'SparseAutoEncoder': 'rgb(31, 119, 180)', 'MLP': 'rgb(255, 127, 14)'}

max_flops_per_model = []

for model_name, model_results in data['results'].items():
    all_flops = []
    all_mccs = []
    
    for run_results in model_results:
        total_flops = [log['total_flops'] for log in run_results]
        mcc = [log['mcc'] for log in run_results]
        
        # Add 0 FLOP-MCC point
        total_flops.insert(0, 0)
        mcc.insert(0, 0)  # Assuming MCC is 0 at the start
        
        all_flops.append(total_flops)
        all_mccs.append(mcc)
    
    # Calculate mean and standard deviation
    mean_flops = np.mean(all_flops, axis=0)
    mean_mccs = np.mean(all_mccs, axis=0)
    std_mccs = np.std(all_mccs, axis=0)
    
    # Store the maximum FLOP value for this model
    max_flops_per_model.append(np.max(mean_flops))
    
    final_mcc = mean_mccs[-1]
    
    # Add trace for each model
    fig.add_trace(
        go.Scatter(
            x=mean_flops,
            y=mean_mccs,
            mode='lines',
            name=f"{model_name} (Final MCC: {final_mcc:.4f})",
            line=dict(color=colors[model_name], width=3),
        )
    )
    
    # Add error bands
    fig.add_trace(
        go.Scatter(
            x=np.concatenate([mean_flops, mean_flops[::-1]]),
            y=np.concatenate([mean_mccs + std_mccs, (mean_mccs - std_mccs)[::-1]]),
            fill='toself',
            fillcolor=colors[model_name].replace('rgb', 'rgba').replace(')', ',0.2)'),
            line=dict(color='rgba(255,255,255,0)'),
            hoverinfo="skip",
            showlegend=False
        )
    )

# Calculate the minimum of the maximum FLOP values
x_max = min(max_flops_per_model)

# Update layout
fig.update_layout(
    xaxis_title='Total FLOPs',
    yaxis_title='MCC',
    font=dict(family="Palatino", size=18),
    legend=dict(
        x=0.01,
        y=0.99,
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='rgba(0, 0, 0, 0.1)',
        borderwidth=1,
        font=dict(size=16)
    ),
    plot_bgcolor='white',
    hovermode='x unified',
    width=800,  # Narrower plot
    height=500,
    margin=dict(l=80, r=50, t=100, b=80)
)

# Update axes
fig.update_xaxes(
    showgrid=True,
    gridwidth=1,
    gridcolor='rgba(0, 0, 0, 0.1)',
    zeroline=False,
    showline=True,
    linewidth=2,
    linecolor='black',
    type='log',  # Use log scale for FLOPs
    title_standoff=20,
    #range=[0, np.log10(x_max)]  # Set x-axis range
)

fig.update_yaxes(
    showgrid=True,
    gridwidth=1,
    gridcolor='rgba(0, 0, 0, 0.1)',
    zeroline=False,
    showline=True,
    linewidth=2,
    linecolor='black',
    range=[0, 1],  # Adjust this range as needed
    title_standoff=20
)

# Save the plot as a PDF
fig.write_image(f"results/figures/{filename}.pdf")

# Optionally, display the plot
fig.show()

print(f"Plot saved as results/figures/{filename}.pdf")

Plot saved as results/figures/fixed_D.pdf


In [45]:
import json
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

FIXED_D = False  # Set this to True or False based on your experiment
filename = 'fixed_D' if FIXED_D else 'learned_D'

# Load the JSON data
with open(f'results/{filename}_results.json', 'r') as f:
    data = json.load(f)

# Create subplots
fig = make_subplots(rows=1, cols=1, shared_xaxes=True)

# Colors for each model
colors = {'SparseAutoEncoder': 'rgb(31, 119, 180)', 'MLP': 'rgb(255, 127, 14)'}

max_flops_per_model = []

for model_name, model_results in data['results'].items():
    all_flops = []
    all_mccs = []
    
    for run_results in model_results:
        total_flops = [log['total_flops'] for log in run_results]
        mcc = [log['mcc'] for log in run_results]
        
        # Add 0 FLOP-MCC point
        total_flops.insert(0, 0)
        mcc.insert(0, 0)  # Assuming MCC is 0 at the start
        
        all_flops.append(total_flops)
        all_mccs.append(mcc)
    
    # Calculate mean and standard deviation
    mean_flops = np.mean(all_flops, axis=0)
    mean_mccs = np.mean(all_mccs, axis=0)
    std_mccs = np.std(all_mccs, axis=0)
    
    # Store the maximum FLOP value for this model
    max_flops_per_model.append(np.max(mean_flops))
    
    final_mcc = mean_mccs[-1]
    
    # Add trace for each model
    fig.add_trace(
        go.Scatter(
            x=mean_flops,
            y=mean_mccs,
            mode='lines',
            name=f"{model_name} (Final MCC: {final_mcc:.4f})",
            line=dict(color=colors[model_name], width=3),
        )
    )
    
    # Add error bands
    fig.add_trace(
        go.Scatter(
            x=np.concatenate([mean_flops, mean_flops[::-1]]),
            y=np.concatenate([mean_mccs + std_mccs, (mean_mccs - std_mccs)[::-1]]),
            fill='toself',
            fillcolor=colors[model_name].replace('rgb', 'rgba').replace(')', ',0.2)'),
            line=dict(color='rgba(255,255,255,0)'),
            hoverinfo="skip",
            showlegend=False
        )
    )

# Calculate the minimum of the maximum FLOP values
x_max = min(max_flops_per_model)

# Update layout
fig.update_layout(
    xaxis_title='Total FLOPs',
    yaxis_title='MCC',
    font=dict(family="Palatino", size=18),
    legend=dict(
        x=0.01,
        y=0.99,
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='rgba(0, 0, 0, 0.1)',
        borderwidth=1,
        font=dict(size=16)
    ),
    plot_bgcolor='white',
    hovermode='x unified',
    width=800,  # Narrower plot
    height=500,
    margin=dict(l=80, r=50, t=100, b=80)
)

# Update axes
fig.update_xaxes(
    showgrid=True,
    gridwidth=1,
    gridcolor='rgba(0, 0, 0, 0.1)',
    zeroline=False,
    showline=True,
    linewidth=2,
    linecolor='black',
    type='log',  # Use log scale for FLOPs
    title_standoff=20,
    #range=[0, np.log10(x_max)]  # Set x-axis range
)

fig.update_yaxes(
    showgrid=True,
    gridwidth=1,
    gridcolor='rgba(0, 0, 0, 0.1)',
    zeroline=False,
    showline=True,
    linewidth=2,
    linecolor='black',
    range=[0, 1],  # Adjust this range as needed
    title_standoff=20
)

# Save the plot as a PDF
fig.write_image(f"results/figures/{filename}.pdf")

# Optionally, display the plot
fig.show()

print(f"Plot saved as results/figures/{filename}.pdf")

Plot saved as results/figures/learned_D.pdf
